In [4]:
import pandas as pd

# 파일 읽기 (파일명 수정해줘)
df = pd.read_csv('지하철혼잡도정보_23년.csv', encoding='cp949')  # 에러나면 encoding='utf-8' 시도

# 1. 전체 컬럼명 출력
print("📌 전체 컬럼명:")
print(df.columns.tolist())

# 2. 데이터 샘플 (처음 3줄)
print("\n📌 데이터 샘플:")
print(df.head(3))

# 3. 데이터 크기
print(f"\n📌 데이터 크기: {df.shape}")  # (행 수, 열 수)




📌 전체 컬럼명:
['연번', '요일구분', '호선', '역번호', '출발역', '상하구분', '5시30분', '6시00분', '6시30분', '7시00분', '7시30분', '8시00분', '8시30분', '9시00분', '9시30분', '10시00분', '10시30분', '11시00분', '11시30분', '12시00분', '12시30분', '13시00분', '13시30분', '14시00분', '14시30분', '15시00분', '15시30분', '16시00분', '16시30분', '17시00분', '17시30분', '18시00분', '18시30분', '19시00분', '19시30분', '20시00분', '20시30분', '21시00분', '21시30분', '22시00분', '22시30분', '23시00분', '23시30분', '00시00분', '00시30분']

📌 데이터 샘플:
   연번 요일구분  호선  역번호  출발역 상하구분  5시30분  6시00분  6시30분  7시00분  ...  20시00분  \
0   1   평일   1  158  청량리   상선    9.1    7.5    4.9    6.9  ...    25.5   
1   2   평일   1  158  청량리   하선   20.4   18.1   26.8   41.8  ...    10.2   
2   3   평일   1  157  제기동   상선    9.0    9.2    6.0    8.0  ...    28.0   

   20시30분  21시00분  21시30분  22시00분  22시30분  23시00분  23시30분  00시00분  00시30분  
0    20.7    27.8    24.4    27.2    27.6    19.6    14.3     4.9     0.0  
1     8.9    10.2     9.2     8.3     7.6    10.8     7.2     0.3     0.2  
2    22.9    33.2    27.8    2

In [6]:
import pandas as pd
import numpy as np

# ============================================
# 1. 데이터 로드
# ============================================
df = pd.read_csv('지하철혼잡도정보_23년.csv', encoding='cp949')

print("=" * 60)
print("📌 원본 데이터 정보")
print("=" * 60)
print(f"데이터 크기: {df.shape}")
print(f"역 개수: {df['출발역'].nunique()}개")
print(f"호선 종류: {df['호선'].unique()}")
print(f"요일 종류: {df['요일구분'].unique()}")
print(f"상하구분: {df['상하구분'].unique()}")

# ============================================
# 2. 시간 컬럼 자동 추출
# ============================================
# '시'와 '분'이 들어간 컬럼만 추출
time_columns = [col for col in df.columns if '시' in col and '분' in col]

print(f"\n📌 시간 컬럼 {len(time_columns)}개 발견:")
print(time_columns[:5], "...", time_columns[-3:])

# ============================================
# 3. Wide → Long 변환 (핵심!)
# ============================================
# 고정할 컬럼 (시간 컬럼 제외한 나머지)
id_vars = ['연번', '요일구분', '호선', '역번호', '출발역', '상하구분']

print(f"\n📌 고정 컬럼: {id_vars}")
print("\n🔄 Wide → Long 변환 중...")

# melt로 변환
df_long = pd.melt(
    df,
    id_vars=id_vars,           # 고정할 컬럼
    value_vars=time_columns,   # 시간 컬럼들
    var_name='시간대',          # 새로운 시간 컬럼명
    value_name='혼잡도'         # 새로운 값 컬럼명
)

print(f"✅ 변환 완료!")
print(f"   변환 전: {df.shape} → 변환 후: {df_long.shape}")
print(f"   {df.shape[0]}개 역 × {len(time_columns)}개 시간 = {df_long.shape[0]}개 행")

# ============================================
# 4. 데이터 전처리
# ============================================
print("\n" + "=" * 60)
print("🔧 데이터 전처리 시작")
print("=" * 60)

# 4-1. 결측치 확인 및 처리
print("\n📌 결측치 개수:")
print(df_long.isnull().sum())

missing_count = df_long['혼잡도'].isnull().sum()
print(f"\n혼잡도 결측치: {missing_count}개 ({missing_count/len(df_long)*100:.2f}%)")

# 결측치 제거
df_long = df_long.dropna(subset=['혼잡도'])
print(f"✅ 결측치 제거 완료: {df_long.shape}")

# 4-2. 혼잡도를 숫자로 변환
df_long['혼잡도'] = pd.to_numeric(df_long['혼잡도'], errors='coerce')

# 4-3. 시간대 정제 (5시30분 → 05:30)
def clean_time(time_str):
    """시간 문자열을 HH:MM 형태로 변환"""
    time_str = str(time_str)
    
    # "5시30분" 형태 처리
    if '시' in time_str and '분' in time_str:
        # "5시30분" → ["5", "30"]
        hour = time_str.split('시')[0].strip().zfill(2)
        minute = time_str.split('시')[1].replace('분', '').strip().zfill(2)
        return f"{hour}:{minute}"
    
    return time_str

df_long['시간대'] = df_long['시간대'].apply(clean_time)

print("\n✅ 시간대 정제 완료")
print(df_long['시간대'].unique()[:10])

# 4-4. 시간대를 숫자로 변환 (정렬용)
df_long['시간_숫자'] = df_long['시간대'].str.replace(':', '').astype(int)

# 4-5. 시간대 순서로 정렬
df_long = df_long.sort_values(['호선', '출발역', '상하구분', '요일구분', '시간_숫자'])
df_long = df_long.reset_index(drop=True)

print("✅ 데이터 정렬 완료")

# ============================================
# 5. 최종 데이터 확인
# ============================================
print("\n" + "=" * 60)
print("📊 전처리 완료 - 최종 데이터 요약")
print("=" * 60)

print(f"\n🔢 데이터 크기: {df_long.shape}")
print(f"📍 고유 역 개수: {df_long['출발역'].nunique()}개")
print(f"🚇 호선 종류: {sorted(df_long['호선'].unique())}")
print(f"📅 요일 종류: {df_long['요일구분'].unique()}")
print(f"⬆️⬇️ 상하구분: {df_long['상하구분'].unique()}")
print(f"⏰ 시간대 개수: {df_long['시간대'].nunique()}개")
print(f"📊 혼잡도 범위: {df_long['혼잡도'].min():.1f} ~ {df_long['혼잡도'].max():.1f}")

print("\n📌 샘플 데이터 (청량리역 평일 상선):")
sample = df_long[
    (df_long['출발역'] == '청량리') & 
    (df_long['요일구분'] == '평일') & 
    (df_long['상하구분'] == '상선')
].head(10)
print(sample[['출발역', '요일구분', '상하구분', '시간대', '혼잡도']])

# ============================================
# 6. 저장
# ============================================
# 전처리된 데이터 저장
output_file = '지하철혼잡도_전처리완료.csv'
df_long.to_csv(output_file, index=False, encoding='cp949')

print(f"\n💾 전처리 완료 파일 저장: '{output_file}'")

# ============================================
# 7. 간단한 통계
# ============================================
print("\n" + "=" * 60)
print("📈 기본 통계")
print("=" * 60)

print(f"\n⏰ 가장 혼잡한 시간대 TOP 5:")
top_time = df_long.groupby('시간대')['혼잡도'].mean().sort_values(ascending=False).head(5)
for time, cong in top_time.items():
    print(f"   {time} : {cong:.1f}")

print(f"\n🚇 가장 혼잡한 역 TOP 5:")
top_station = df_long.groupby('출발역')['혼잡도'].mean().sort_values(ascending=False).head(5)
for station, cong in top_station.items():
    print(f"   {station} : {cong:.1f}")

print("\n" + "=" * 60)
print("✅ 전처리 완료! 다음 단계(EDA) 준비 완료!")
print("=" * 60)

📌 원본 데이터 정보
데이터 크기: (1668, 45)
역 개수: 243개
호선 종류: [1 2 3 4 5 6 7 8]
요일 종류: ['평일' '토요일' '일요일']
상하구분: ['상선' '하선' '내선' '외선']

📌 시간 컬럼 39개 발견:
['5시30분', '6시00분', '6시30분', '7시00분', '7시30분'] ... ['23시30분', '00시00분', '00시30분']

📌 고정 컬럼: ['연번', '요일구분', '호선', '역번호', '출발역', '상하구분']

🔄 Wide → Long 변환 중...
✅ 변환 완료!
   변환 전: (1668, 45) → 변환 후: (65052, 8)
   1668개 역 × 39개 시간 = 65052개 행

🔧 데이터 전처리 시작

📌 결측치 개수:
연번      0
요일구분    0
호선      0
역번호     0
출발역     0
상하구분    0
시간대     0
혼잡도     0
dtype: int64

혼잡도 결측치: 0개 (0.00%)
✅ 결측치 제거 완료: (65052, 8)

✅ 시간대 정제 완료
['05:30' '06:00' '06:30' '07:00' '07:30' '08:00' '08:30' '09:00' '09:30'
 '10:00']
✅ 데이터 정렬 완료

📊 전처리 완료 - 최종 데이터 요약

🔢 데이터 크기: (65052, 9)
📍 고유 역 개수: 243개
🚇 호선 종류: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8)]
📅 요일 종류: ['일요일' '토요일' '평일']
⬆️⬇️ 상하구분: ['상선' '하선' '내선' '외선']
⏰ 시간대 개수: 39개
📊 혼잡도 범위: 0.0 ~ 190.2

📌 샘플 데이터 (청량리역 평일 상선):
      출발역 요일구분 상하구분    시간대   혼잡도
2184  청량리   평일   상선  00:00 

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager, rc

# ============================================
# 0. 한글 폰트 설정 (그래프 한글 깨짐 방지)
# ============================================
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows
# Mac이면: plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

# ============================================
# 1. 데이터 로드
# ============================================
df = pd.read_csv('지하철혼잡도_전처리완료.csv', encoding='cp949')

print("=" * 60)
print("📊 지하철 혼잡도 종합 분석 시작")
print("=" * 60)
print(f"데이터 크기: {df.shape}")
print(f"분석 기간: 2023년")
print(f"총 {df['출발역'].nunique()}개 역 × {df['시간대'].nunique()}개 시간대")
print("=" * 60)

# ============================================
# 2. 시각화 설정
# ============================================
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (15, 10)

# 전체 그래프를 담을 Figure 생성
fig = plt.figure(figsize=(20, 24))

# ============================================
# 📈 분석 1: 시간대별 평균 혼잡도 (가장 중요!)
# ============================================
print("\n[1/10] 시간대별 평균 혼잡도 분석 중...")

ax1 = plt.subplot(5, 2, 1)
time_avg = df.groupby('시간대')['혼잡도'].mean().sort_index()

# 시간대 순서 정렬을 위해 숫자로 변환
time_order = sorted(time_avg.index, key=lambda x: int(x.replace(':', '')))
time_avg_sorted = time_avg.reindex(time_order)

plt.plot(range(len(time_avg_sorted)), time_avg_sorted.values, 
         marker='o', linewidth=2, markersize=4, color='#2E86AB')
plt.fill_between(range(len(time_avg_sorted)), time_avg_sorted.values, 
                 alpha=0.3, color='#2E86AB')
plt.xticks(range(0, len(time_avg_sorted), 4), 
           [time_order[i] for i in range(0, len(time_order), 4)], 
           rotation=45)
plt.xlabel('시간대', fontsize=10)
plt.ylabel('평균 혼잡도', fontsize=10)
plt.title('📈 시간대별 평균 혼잡도', fontsize=12, fontweight='bold')
plt.grid(True, alpha=0.3)

# 피크 시간 표시
peak_idx = time_avg_sorted.idxmax()
peak_value = time_avg_sorted.max()
peak_position = list(time_avg_sorted.index).index(peak_idx)
plt.axvline(x=peak_position, color='red', linestyle='--', alpha=0.5, linewidth=2)
plt.text(peak_position, peak_value, f'  피크\n  {peak_idx}\n  {peak_value:.1f}', 
         fontsize=9, color='red', fontweight='bold')

# ============================================
# 📊 분석 2: 요일별 혼잡도 비교
# ============================================
print("[2/10] 요일별 혼잡도 분석 중...")

ax2 = plt.subplot(5, 2, 2)
day_avg = df.groupby('요일구분')['혼잡도'].mean().sort_values(ascending=False)

colors = ['#E63946' if x == day_avg.max() else '#457B9D' for x in day_avg.values]
bars = plt.bar(range(len(day_avg)), day_avg.values, color=colors, edgecolor='black', linewidth=1.5)
plt.xticks(range(len(day_avg)), day_avg.index, fontsize=10)
plt.ylabel('평균 혼잡도', fontsize=10)
plt.title('📅 요일별 평균 혼잡도', fontsize=12, fontweight='bold')
plt.grid(axis='y', alpha=0.3)

# 값 표시
for i, (bar, val) in enumerate(zip(bars, day_avg.values)):
    plt.text(i, val + 1, f'{val:.1f}', ha='center', fontsize=9, fontweight='bold')

# ============================================
# 🚇 분석 3: 역별 혼잡도 TOP 10
# ============================================
print("[3/10] 역별 혼잡도 랭킹 분석 중...")

ax3 = plt.subplot(5, 2, 3)
station_avg = df.groupby('출발역')['혼잡도'].mean().sort_values(ascending=False).head(10)

plt.barh(range(len(station_avg)), station_avg.values, color='#F4A261', edgecolor='black', linewidth=1.5)
plt.yticks(range(len(station_avg)), station_avg.index, fontsize=9)
plt.xlabel('평균 혼잡도', fontsize=10)
plt.title('🏆 가장 혼잡한 역 TOP 10', fontsize=12, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)

# 값 표시
for i, val in enumerate(station_avg.values):
    plt.text(val + 0.5, i, f'{val:.1f}', va='center', fontsize=8)

# ============================================
# 🚇 분석 4: 역별 혼잡도 BOTTOM 10
# ============================================
print("[4/10] 한산한 역 분석 중...")

ax4 = plt.subplot(5, 2, 4)
station_low = df.groupby('출발역')['혼잡도'].mean().sort_values(ascending=True).head(10)

plt.barh(range(len(station_low)), station_low.values, color='#06D6A0', edgecolor='black', linewidth=1.5)
plt.yticks(range(len(station_low)), station_low.index, fontsize=9)
plt.xlabel('평균 혼잡도', fontsize=10)
plt.title('😌 가장 한산한 역 TOP 10', fontsize=12, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)

# 값 표시
for i, val in enumerate(station_low.values):
    plt.text(val + 0.5, i, f'{val:.1f}', va='center', fontsize=8)

# ============================================
# 🚉 분석 5: 호선별 혼잡도
# ============================================
print("[5/10] 호선별 혼잡도 분석 중...")

ax5 = plt.subplot(5, 2, 5)
line_avg = df.groupby('호선')['혼잡도'].mean().sort_values(ascending=False)

colors_line = plt.cm.tab10(range(len(line_avg)))
bars = plt.bar(range(len(line_avg)), line_avg.values, color=colors_line, edgecolor='black', linewidth=1.5)
plt.xticks(range(len(line_avg)), line_avg.index, fontsize=10)
plt.ylabel('평균 혼잡도', fontsize=10)
plt.title('🚇 호선별 평균 혼잡도', fontsize=12, fontweight='bold')
plt.grid(axis='y', alpha=0.3)

# 값 표시
for i, (bar, val) in enumerate(zip(bars, line_avg.values)):
    plt.text(i, val + 0.5, f'{val:.1f}', ha='center', fontsize=9, fontweight='bold')

# ============================================
# ⬆️⬇️ 분석 6: 상하선 비교
# ============================================
print("[6/10] 상하선 혼잡도 비교 중...")

ax6 = plt.subplot(5, 2, 6)
direction_avg = df.groupby('상하구분')['혼잡도'].mean()

colors_dir = ['#E76F51', '#2A9D8F']
bars = plt.bar(range(len(direction_avg)), direction_avg.values, 
               color=colors_dir, edgecolor='black', linewidth=1.5)
plt.xticks(range(len(direction_avg)), direction_avg.index, fontsize=11)
plt.ylabel('평균 혼잡도', fontsize=10)
plt.title('⬆️⬇️ 상하선 평균 혼잡도', fontsize=12, fontweight='bold')
plt.grid(axis='y', alpha=0.3)

# 값 표시
for i, (bar, val) in enumerate(zip(bars, direction_avg.values)):
    plt.text(i, val + 1, f'{val:.1f}', ha='center', fontsize=11, fontweight='bold')

# ============================================
# 🕐 분석 7: 요일별 시간대 패턴
# ============================================
print("[7/10] 요일별 시간대 패턴 분석 중...")

ax7 = plt.subplot(5, 2, 7)
pivot_day_time = df.groupby(['요일구분', '시간_숫자'])['혼잡도'].mean().unstack(level=0)

for col in pivot_day_time.columns:
    plt.plot(pivot_day_time.index, pivot_day_time[col], marker='o', 
             markersize=3, linewidth=1.5, label=col, alpha=0.8)

plt.xlabel('시간대', fontsize=10)
plt.ylabel('평균 혼잡도', fontsize=10)
plt.title('📊 요일별 시간대 혼잡도 패턴', fontsize=12, fontweight='bold')
plt.legend(fontsize=9, loc='upper left')
plt.grid(True, alpha=0.3)

# x축 라벨을 시간 형식으로
time_labels = sorted(df['시간대'].unique(), key=lambda x: int(x.replace(':', '')))
xtick_positions = sorted(df['시간_숫자'].unique())
plt.xticks(xtick_positions[::4], [time_labels[i] for i in range(0, len(time_labels), 4)], 
           rotation=45, fontsize=8)

# ============================================
# 🎯 분석 8: 피크시간대 상하선 비교
# ============================================
print("[8/10] 피크시간 상하선 패턴 분석 중...")

ax8 = plt.subplot(5, 2, 8)

# 출근시간(07:00~09:00) vs 퇴근시간(17:00~19:00)
morning_peak = df[df['시간_숫자'].between(700, 900)]
evening_peak = df[df['시간_숫자'].between(1700, 1900)]

morning_dir = morning_peak.groupby('상하구분')['혼잡도'].mean()
evening_dir = evening_peak.groupby('상하구분')['혼잡도'].mean()

x = np.arange(len(morning_dir.index))
width = 0.35

bars1 = plt.bar(x - width/2, morning_dir.values, width, 
                label='출근시간(07~09시)', color='#FFB703', edgecolor='black', linewidth=1.5)
bars2 = plt.bar(x + width/2, evening_dir.values, width, 
                label='퇴근시간(17~19시)', color='#FB8500', edgecolor='black', linewidth=1.5)

plt.xticks(x, morning_dir.index, fontsize=10)
plt.ylabel('평균 혼잡도', fontsize=10)
plt.title('🕐 피크시간대 상하선 비교', fontsize=12, fontweight='bold')
plt.legend(fontsize=9)
plt.grid(axis='y', alpha=0.3)

# 값 표시
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 1,
                f'{height:.1f}', ha='center', fontsize=9, fontweight='bold')

# ============================================
# 📦 분석 9: 혼잡도 분포 (박스플롯)
# ============================================
print("[9/10] 혼잡도 분포 분석 중...")

ax9 = plt.subplot(5, 2, 9)

# 호선별 박스플롯
box_data = [df[df['호선'] == line]['혼잡도'].values for line in sorted(df['호선'].unique())]
bp = plt.boxplot(box_data, labels=sorted(df['호선'].unique()), 
                 patch_artist=True, showfliers=False)

# 박스 색상
for patch, color in zip(bp['boxes'], plt.cm.tab10(range(len(box_data)))):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

plt.xlabel('호선', fontsize=10)
plt.ylabel('혼잡도', fontsize=10)
plt.title('📦 호선별 혼잡도 분포', fontsize=12, fontweight='bold')
plt.grid(axis='y', alpha=0.3)

# ============================================
# 🔥 분석 10: 히트맵 - 호선별 시간대 혼잡도
# ============================================
print("[10/10] 히트맵 생성 중...")

ax10 = plt.subplot(5, 2, 10)

# 호선별 × 시간대별 평균 혼잡도
pivot_heat = df.groupby(['호선', '시간_숫자'])['혼잡도'].mean().unstack(level=0).T

# 시간대 라벨 생성 (4시간 간격)
time_labels_heat = sorted(df['시간대'].unique(), key=lambda x: int(x.replace(':', '')))
selected_times = time_labels_heat[::4]  # 4시간 간격
selected_indices = [i for i in range(0, len(time_labels_heat), 4)]

sns.heatmap(pivot_heat, cmap='YlOrRd', annot=False, fmt='.0f', 
            cbar_kws={'label': '혼잡도'}, linewidths=0.5)
plt.xlabel('시간대', fontsize=10)
plt.ylabel('호선', fontsize=10)
plt.title('🔥 호선별 × 시간대별 혼잡도 히트맵', fontsize=12, fontweight='bold')

# x축 라벨 설정
current_xticks = plt.xticks()[0]
plt.xticks(current_xticks[::4], selected_times, rotation=45, fontsize=8)

# ============================================
# 전체 레이아웃 조정 및 저장
# ============================================
plt.tight_layout()
plt.savefig('지하철혼잡도_종합분석.png', dpi=300, bbox_inches='tight')
print("\n✅ 그래프 저장 완료: '지하철혼잡도_종합분석.png'")

plt.show()

# ============================================
# 📊 핵심 인사이트 출력
# ============================================
print("\n" + "=" * 60)
print("🔍 핵심 인사이트 요약")
print("=" * 60)

print(f"\n⏰ 가장 혼잡한 시간대: {time_avg_sorted.idxmax()} ({time_avg_sorted.max():.1f})")
print(f"😌 가장 한산한 시간대: {time_avg_sorted.idxmin()} ({time_avg_sorted.min():.1f})")

print(f"\n🚇 가장 혼잡한 역: {station_avg.index[0]} ({station_avg.values[0]:.1f})")
print(f"😌 가장 한산한 역: {station_low.index[0]} ({station_low.values[0]:.1f})")

print(f"\n🚉 가장 혼잡한 호선: {line_avg.index[0]} ({line_avg.values[0]:.1f})")
print(f"😌 가장 한산한 호선: {line_avg.index[-1]} ({line_avg.values[-1]:.1f})")

print(f"\n📅 가장 혼잡한 요일: {day_avg.index[0]} ({day_avg.values[0]:.1f})")

print(f"\n⬆️⬇️ 상하선 비교:")
for direction, value in direction_avg.items():
    print(f"   {direction}: {value:.1f}")

print(f"\n🕐 출근시간(07~09시) 상하선:")
for direction, value in morning_dir.items():
    print(f"   {direction}: {value:.1f}")

print(f"\n🕐 퇴근시간(17~19시) 상하선:")
for direction, value in evening_dir.items():
    print(f"   {direction}: {value:.1f}")

print("\n" + "=" * 60)
print("✅ 종합 분석 완료!")
print("=" * 60)

ModuleNotFoundError: No module named 'matplotlib'

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# ============================================
# 1. 데이터 로드
# ============================================
df = pd.read_csv('지하철혼잡도_전처리완료.csv', encoding='cp949')

print("=" * 60)
print("📊 지하철 혼잡도 종합 분석 시작 (Plotly 버전)")
print("=" * 60)
print(f"데이터 크기: {df.shape}")
print(f"분석 기간: 2023년")
print(f"총 {df['출발역'].nunique()}개 역 × {df['시간대'].nunique()}개 시간대")
print("=" * 60)

# ============================================
# 2. 서브플롯 생성 (5행 2열 = 10개 그래프)
# ============================================
fig = make_subplots(
    rows=5, cols=2,
    subplot_titles=(
        '📈 시간대별 평균 혼잡도',
        '📅 요일별 평균 혼잡도',
        '🏆 가장 혼잡한 역 TOP 10',
        '😌 가장 한산한 역 TOP 10',
        '🚇 호선별 평균 혼잡도',
        '⬆️⬇️ 상하선 평균 혼잡도',
        '📊 요일별 시간대 혼잡도 패턴',
        '🕐 피크시간대 상하선 비교',
        '📦 호선별 혼잡도 분포',
        '🔥 호선별 × 시간대별 히트맵'
    ),
    specs=[
        [{"type": "scatter"}, {"type": "bar"}],
        [{"type": "bar"}, {"type": "bar"}],
        [{"type": "bar"}, {"type": "bar"}],
        [{"type": "scatter"}, {"type": "bar"}],
        [{"type": "box"}, {"type": "heatmap"}]
    ],
    vertical_spacing=0.08,
    horizontal_spacing=0.12
)

# ============================================
# 📈 분석 1: 시간대별 평균 혼잡도
# ============================================
print("\n[1/10] 시간대별 평균 혼잡도 분석 중...")

time_avg = df.groupby('시간대')['혼잡도'].mean().sort_index()
time_order = sorted(time_avg.index, key=lambda x: int(x.replace(':', '')))
time_avg_sorted = time_avg.reindex(time_order)

fig.add_trace(
    go.Scatter(
        x=time_order,
        y=time_avg_sorted.values,
        mode='lines+markers',
        name='시간대별 혼잡도',
        line=dict(color='#2E86AB', width=3),
        marker=dict(size=6),
        fill='tozeroy',
        fillcolor='rgba(46, 134, 171, 0.3)',
        hovertemplate='시간: %{x}<br>혼잡도: %{y:.1f}<extra></extra>'
    ),
    row=1, col=1
)

# 피크 시간 표시
peak_idx = time_avg_sorted.idxmax()
peak_value = time_avg_sorted.max()
fig.add_annotation(
    x=peak_idx, y=peak_value,
    text=f"피크<br>{peak_idx}<br>{peak_value:.1f}",
    showarrow=True,
    arrowhead=2,
    arrowcolor="red",
    font=dict(color="red", size=10),
    row=1, col=1
)

# ============================================
# 📅 분석 2: 요일별 혼잡도
# ============================================
print("[2/10] 요일별 혼잡도 분석 중...")

day_avg = df.groupby('요일구분')['혼잡도'].mean().sort_values(ascending=False)

colors = ['#E63946' if x == day_avg.max() else '#457B9D' for x in day_avg.values]

fig.add_trace(
    go.Bar(
        x=list(day_avg.index),
        y=day_avg.values,
        marker=dict(color=colors, line=dict(color='black', width=1.5)),
        text=[f'{v:.1f}' for v in day_avg.values],
        textposition='outside',
        name='요일별 혼잡도',
        hovertemplate='%{x}<br>혼잡도: %{y:.1f}<extra></extra>'
    ),
    row=1, col=2
)

# ============================================
# 🚇 분석 3: 역별 혼잡도 TOP 10
# ============================================
print("[3/10] 역별 혼잡도 랭킹 분석 중...")

station_avg = df.groupby('출발역')['혼잡도'].mean().sort_values(ascending=False).head(10)

fig.add_trace(
    go.Bar(
        y=list(station_avg.index)[::-1],
        x=station_avg.values[::-1],
        orientation='h',
        marker=dict(color='#F4A261', line=dict(color='black', width=1.5)),
        text=[f'{v:.1f}' for v in station_avg.values[::-1]],
        textposition='outside',
        name='혼잡한 역',
        hovertemplate='%{y}<br>혼잡도: %{x:.1f}<extra></extra>'
    ),
    row=2, col=1
)

# ============================================
# 😌 분석 4: 역별 혼잡도 BOTTOM 10
# ============================================
print("[4/10] 한산한 역 분석 중...")

station_low = df.groupby('출발역')['혼잡도'].mean().sort_values(ascending=True).head(10)

fig.add_trace(
    go.Bar(
        y=list(station_low.index)[::-1],
        x=station_low.values[::-1],
        orientation='h',
        marker=dict(color='#06D6A0', line=dict(color='black', width=1.5)),
        text=[f'{v:.1f}' for v in station_low.values[::-1]],
        textposition='outside',
        name='한산한 역',
        hovertemplate='%{y}<br>혼잡도: %{x:.1f}<extra></extra>'
    ),
    row=2, col=2
)

# ============================================
# 🚉 분석 5: 호선별 혼잡도
# ============================================
print("[5/10] 호선별 혼잡도 분석 중...")

line_avg = df.groupby('호선')['혼잡도'].mean().sort_values(ascending=False)

fig.add_trace(
    go.Bar(
        x=list(line_avg.index),
        y=line_avg.values,
        marker=dict(
            color=line_avg.values,
            colorscale='Viridis',
            line=dict(color='black', width=1.5)
        ),
        text=[f'{v:.1f}' for v in line_avg.values],
        textposition='outside',
        name='호선별 혼잡도',
        hovertemplate='%{x}호선<br>혼잡도: %{y:.1f}<extra></extra>'
    ),
    row=3, col=1
)

# ============================================
# ⬆️⬇️ 분석 6: 상하선 비교
# ============================================
print("[6/10] 상하선 혼잡도 비교 중...")

direction_avg = df.groupby('상하구분')['혼잡도'].mean()

fig.add_trace(
    go.Bar(
        x=list(direction_avg.index),
        y=direction_avg.values,
        marker=dict(color=['#E76F51', '#2A9D8F'], line=dict(color='black', width=1.5)),
        text=[f'{v:.1f}' for v in direction_avg.values],
        textposition='outside',
        name='상하선 혼잡도',
        hovertemplate='%{x}<br>혼잡도: %{y:.1f}<extra></extra>'
    ),
    row=3, col=2
)

# ============================================
# 📊 분석 7: 요일별 시간대 패턴
# ============================================
print("[7/10] 요일별 시간대 패턴 분석 중...")

pivot_day_time = df.groupby(['요일구분', '시간_숫자'])['혼잡도'].mean().unstack(level=0)

colors_day = px.colors.qualitative.Set2
for i, col in enumerate(pivot_day_time.columns):
    fig.add_trace(
        go.Scatter(
            x=pivot_day_time.index,
            y=pivot_day_time[col],
            mode='lines+markers',
            name=col,
            line=dict(width=2, color=colors_day[i % len(colors_day)]),
            marker=dict(size=4),
            hovertemplate=f'{col}<br>시간: %{{x}}<br>혼잡도: %{{y:.1f}}<extra></extra>'
        ),
        row=4, col=1
    )

# ============================================
# 🕐 분석 8: 피크시간대 상하선 비교
# ============================================
print("[8/10] 피크시간 상하선 패턴 분석 중...")

morning_peak = df[df['시간_숫자'].between(700, 900)]
evening_peak = df[df['시간_숫자'].between(1700, 1900)]

morning_dir = morning_peak.groupby('상하구분')['혼잡도'].mean()
evening_dir = evening_peak.groupby('상하구분')['혼잡도'].mean()

fig.add_trace(
    go.Bar(
        x=list(morning_dir.index),
        y=morning_dir.values,
        name='출근시간(07~09시)',
        marker=dict(color='#FFB703', line=dict(color='black', width=1.5)),
        text=[f'{v:.1f}' for v in morning_dir.values],
        textposition='outside',
        hovertemplate='%{x}<br>출근시간: %{y:.1f}<extra></extra>'
    ),
    row=4, col=2
)

fig.add_trace(
    go.Bar(
        x=list(evening_dir.index),
        y=evening_dir.values,
        name='퇴근시간(17~19시)',
        marker=dict(color='#FB8500', line=dict(color='black', width=1.5)),
        text=[f'{v:.1f}' for v in evening_dir.values],
        textposition='outside',
        hovertemplate='%{x}<br>퇴근시간: %{y:.1f}<extra></extra>'
    ),
    row=4, col=2
)

# ============================================
# 📦 분석 9: 혼잡도 분포 (박스플롯)
# ============================================
print("[9/10] 혼잡도 분포 분석 중...")

for line in sorted(df['호선'].unique()):
    line_data = df[df['호선'] == line]['혼잡도']
    fig.add_trace(
        go.Box(
            y=line_data,
            name=f'{line}호선',
            marker=dict(color=px.colors.qualitative.Set1[line % 9]),
            hovertemplate=f'{line}호선<br>혼잡도: %{{y:.1f}}<extra></extra>'
        ),
        row=5, col=1
    )

# ============================================
# 🔥 분석 10: 히트맵
# ============================================
print("[10/10] 히트맵 생성 중...")

pivot_heat = df.groupby(['호선', '시간_숫자'])['혼잡도'].mean().unstack(level=0).T

# 시간대 라벨 (4시간 간격)
time_labels_heat = sorted(df['시간대'].unique(), key=lambda x: int(x.replace(':', '')))
selected_times = time_labels_heat[::4]

fig.add_trace(
    go.Heatmap(
        z=pivot_heat.values,
        x=pivot_heat.columns,
        y=list(pivot_heat.index),
        colorscale='YlOrRd',
        hovertemplate='호선: %{y}<br>시간: %{x}<br>혼잡도: %{z:.1f}<extra></extra>',
        colorbar=dict(title="혼잡도", x=1.15)
    ),
    row=5, col=2
)

# ============================================
# 레이아웃 설정
# ============================================
fig.update_layout(
    height=2400,
    width=1600,
    title_text="🚇 지하철 혼잡도 종합 분석 대시보드",
    title_font_size=24,
    showlegend=False,
    font=dict(family="Malgun Gothic", size=10)
)

# x축, y축 라벨 설정
fig.update_xaxes(title_text="시간대", row=1, col=1)
fig.update_yaxes(title_text="평균 혼잡도", row=1, col=1)

fig.update_xaxes(title_text="요일", row=1, col=2)
fig.update_yaxes(title_text="평균 혼잡도", row=1, col=2)

fig.update_xaxes(title_text="평균 혼잡도", row=2, col=1)
fig.update_xaxes(title_text="평균 혼잡도", row=2, col=2)

fig.update_xaxes(title_text="호선", row=3, col=1)
fig.update_yaxes(title_text="평균 혼잡도", row=3, col=1)

fig.update_xaxes(title_text="상하구분", row=3, col=2)
fig.update_yaxes(title_text="평균 혼잡도", row=3, col=2)

fig.update_xaxes(title_text="시간대(숫자)", row=4, col=1)
fig.update_yaxes(title_text="평균 혼잡도", row=4, col=1)

fig.update_xaxes(title_text="상하구분", row=4, col=2)
fig.update_yaxes(title_text="평균 혼잡도", row=4, col=2)

fig.update_xaxes(title_text="호선", row=5, col=1)
fig.update_yaxes(title_text="혼잡도", row=5, col=1)

fig.update_xaxes(title_text="시간대", row=5, col=2)
fig.update_yaxes(title_text="호선", row=5, col=2)

# ============================================
# HTML로 저장 (인터랙티브!)
# ============================================
output_html = '지하철혼잡도_종합분석_interactive.html'
fig.write_html(output_html)
print(f"\n✅ 인터랙티브 HTML 저장 완료: '{output_html}'")
print("   👉 브라우저에서 열어서 확대/축소/hover 가능!")

# 정적 이미지로도 저장 (선택사항)
# fig.write_image('지하철혼잡도_plotly.png', width=1600, height=2400)

# 브라우저에서 바로 열기
fig.show()

# ============================================
# 📊 핵심 인사이트 출력
# ============================================
print("\n" + "=" * 60)
print("🔍 핵심 인사이트 요약")
print("=" * 60)

time_avg_sorted = df.groupby('시간대')['혼잡도'].mean().sort_index()
time_order = sorted(time_avg_sorted.index, key=lambda x: int(x.replace(':', '')))
time_avg_sorted = time_avg_sorted.reindex(time_order)

print(f"\n⏰ 가장 혼잡한 시간대: {time_avg_sorted.idxmax()} ({time_avg_sorted.max():.1f})")
print(f"😌 가장 한산한 시간대: {time_avg_sorted.idxmin()} ({time_avg_sorted.min():.1f})")

print(f"\n🚇 가장 혼잡한 역: {station_avg.index[0]} ({station_avg.values[0]:.1f})")
print(f"😌 가장 한산한 역: {station_low.index[0]} ({station_low.values[0]:.1f})")

print(f"\n🚉 가장 혼잡한 호선: {line_avg.index[0]} ({line_avg.values[0]:.1f})")
print(f"😌 가장 한산한 호선: {line_avg.index[-1]} ({line_avg.values[-1]:.1f})")

print(f"\n📅 가장 혼잡한 요일: {day_avg.index[0]} ({day_avg.values[0]:.1f})")

print(f"\n⬆️⬇️ 상하선 비교:")
for direction, value in direction_avg.items():
    print(f"   {direction}: {value:.1f}")

print("\n" + "=" * 60)
print("✅ Plotly 종합 분석 완료!")
print("=" * 60)

📊 지하철 혼잡도 종합 분석 시작 (Plotly 버전)
데이터 크기: (65052, 9)
분석 기간: 2023년
총 243개 역 × 39개 시간대

[1/10] 시간대별 평균 혼잡도 분석 중...
[2/10] 요일별 혼잡도 분석 중...
[3/10] 역별 혼잡도 랭킹 분석 중...
[4/10] 한산한 역 분석 중...
[5/10] 호선별 혼잡도 분석 중...
[6/10] 상하선 혼잡도 비교 중...
[7/10] 요일별 시간대 패턴 분석 중...
[8/10] 피크시간 상하선 패턴 분석 중...
[9/10] 혼잡도 분포 분석 중...
[10/10] 히트맵 생성 중...

✅ 인터랙티브 HTML 저장 완료: '지하철혼잡도_종합분석_interactive.html'
   👉 브라우저에서 열어서 확대/축소/hover 가능!



🔍 핵심 인사이트 요약

⏰ 가장 혼잡한 시간대: 18:00 (40.6)
😌 가장 한산한 시간대: 00:30 (1.6)

🚇 가장 혼잡한 역: 송파 (46.9)
😌 가장 한산한 역: 장암 (0.9)

🚉 가장 혼잡한 호선: 2 (30.9)
😌 가장 한산한 호선: 6 (22.7)

📅 가장 혼잡한 요일: 평일 (31.6)

⬆️⬇️ 상하선 비교:
   내선: 30.5
   상선: 25.6
   외선: 31.2
   하선: 25.2

✅ Plotly 종합 분석 완료!
